In [21]:
import numpy as np
import pandas as pd
import pickle


with open("F:/ESO/Data/MUSE/muse_df.pickle", 'rb') as handle:
    muse_df = pickle.load(handle)

D = 8 # [m], M1 diameter

rad2mas  = 3600 * 180 * 1000 / np.pi
rad2arc  = rad2mas / 1000
deg2rad  = np.pi / 180
asec2rad = np.pi / 180 / 3600

seeing = lambda r0, lmbd: rad2arc*0.976*lmbd/r0 # [arcs]
r0_new = lambda r0, lmbd, lmbd0: r0*(lmbd/lmbd0)**1.2 # [m]
r0 = lambda seeing, lmbd: rad2arc*0.976*lmbd/seeing # [m]

    # from tools.utils import rad2arc, r0_new
WFS_wvl = (1.215e-6 + 1.625e-6) / 2 # IRLOS works in J+H band, so average wavelength is used
WFS_excessive_factor = 1
WFS_d_sub = D / 2 # Since 2 subaps per dimension.


muse_df.tail()

,name,time,window,scale,frequency,gain,"plate scale, [mas/pix]","conversion, [e-/ADU]","RON, [e-]",ADUs (header),...,Surface layer profile [10**(-15)m**(1/3)],"Seeing [""]",Filename,Bad quality,Corrupted,Good quality,Has streaks,Low AO errs,Medium quality,Non-point
ID,,,,,,,,,,,,,,,,,,,,,
407,M.MUSE.2024-04-16T19-39-37.183.fits,2024-04-11 06:36:33.347000+00:00,20.0,Small,500.0,68.0,78.0,9.8,1.0,95996.0,...,NaN,NaN,M.MUSE.2024-04-16T19-39-37.183,False,False,True,True,True,False,True
408,M.MUSE.2024-04-19T04-53-39.793.fits,2024-04-18 06:02:33.553000+00:00,20.0,Small,500.0,68.0,78.0,9.8,1.0,93684.0,...,NaN,NaN,M.MUSE.2024-04-19T04-53-39.793,False,False,True,False,True,False,True
409,M.MUSE.2024-05-02T22-51-44.900.fits,2024-05-01 03:16:40.639000+00:00,20.0,Small,500.0,68.0,78.0,9.8,1.0,7284.0,...,NaN,NaN,M.MUSE.2024-05-02T22-51-44.900,True,False,False,False,False,True,False
410,M.MUSE.2024-05-06T16-44-36.486.fits,2024-05-02 09:43:08.639000+00:00,20.0,Small,500.0,68.0,78.0,9.8,1.0,44932.0,...,NaN,NaN,M.MUSE.2024-05-06T16-44-36.486,False,False,True,True,False,False,False
411,M.MUSE.2024-05-06T16-57-21.086.fits,2024-05-02 23:37:42.444000+00:00,20.0,Small,500.0,68.0,78.0,9.8,1.0,108676.0,...,NaN,NaN,M.MUSE.2024-05-06T16-57-21.086,False,False,True,True,False,False,False


Note that 'window', 'scale', 'frequency', 'gain', 'plate scale, [mas/pix]', 'conversion, [e-/ADU]', 'RON, [e-]', 'ADUs (header)' entries are related to IRLOS.


In [22]:
a = [3.2, 3.1, 3.2, 3.7, 4.4, 5.2, 4.9, 6, 7.7, 8.9, 10, 11.8, 13]
a = np.array(a) * 5

print(2.8*2)
print(a.mean())

muse_df.columns

5.6
32.73076923076923


Index(['name', 'time', 'window', 'scale', 'frequency', 'gain',
       'plate scale, [mas/pix]', 'conversion, [e-/ADU]', 'RON, [e-]',
       'ADUs (header)',
       ...
       'Surface layer profile [10**(-15)m**(1/3)]', 'Seeing ["]', 'Filename',
       'Bad quality', 'Corrupted', 'Good quality', 'Has streaks',
       'Low AO errs', 'Medium quality', 'Non-point'],
      dtype='object', length=188)

What this code does:

1) Computes WFSing noise variance for $2 \times 2$ WFS
2) Then, $\text{OPD}_\text{TT}$ is computed from the variance for the atmospheric wavelength (500 [nm])
3) $\text{OPD}_\text{TT}$ basically tells how much WF error is induced by TT. Then, TT WFE can be recomputed into the angular shift 

* Yes, IRLOS also controls defocus. For simplicity, it is not considered here. But indeed, defocus should also introduce some error apart from TT;
* Note, that IRLOS parameters before and after upgrade differ;
* WFSing noise considered to be the same fopr all sub-apertures;
* Jitter for tip and tilt modes is considered to be the same;

In [5]:
def NoiseVarianceIRLOS(r0, WFS_nPix, WFS_Nph, WFS_psInMas, WFS_RON):

    r0_WFS =  r0_new(r0, WFS_wvl, 0.5e-6)
    WFS_pixelScale = WFS_psInMas / 1e3 # [arcsec]
        
    # Read-out noise calculation
    nD = np.maximum(rad2arc*WFS_wvl/WFS_d_sub/WFS_pixelScale, 1.0)
    # Photon-noise calculation
    nT = np.maximum(rad2arc*WFS_wvl/r0_WFS / WFS_pixelScale, 1.0)

    varRON  = np.pi**2/3 * (WFS_RON**2/WFS_Nph**2) * (WFS_nPix**2/nD)**2
    varShot = np.pi**2 / (2*WFS_Nph) * (nT/nD)**2
    # Noise variance calculation
    noise_variance = WFS_excessive_factor * (varRON+varShot) * (500e-9/WFS_wvl)**2 # Recompute to the wavelength of 500 nm from the WFSing wavelength

    return noise_variance # [rad^2]


def get_TT_jitter(r0, WFS_nPix, WFS_Nph, WFS_psInMas, WFS_RON):
    from tools.utils import rad2mas
    TT_var = NoiseVarianceIRLOS(r0, WFS_nPix, WFS_Nph, WFS_psInMas, WFS_RON)

    TT_OPD = np.sqrt(TT_var) * 0.5e-6 / 2 / np.pi # [nm]

    TT_max = 1.9665198 # Value at the edge of tip/tilt mode
    jitter_STD = lambda TT_WFE: np.arctan(2*TT_max/D * TT_WFE) * rad2mas # [mas], TT_WFE in [m]

    return jitter_STD(TT_OPD) # [mas]


def get_jitter_from_ids(ids):
    from tools.utils import r0 as r0_from_seeing
    get_entry = lambda entry, ids: np.array( muse_df[entry].loc[ids].tolist() )

    r0      = r0_from_seeing(get_entry('Seeing (header)', ids), 500e-9)
    IR_ph   = get_entry('IRLOS photons, [photons/s/m^2]', ids)
    IR_win  = get_entry('window', ids)
    IR_freq = get_entry('frequency', ids)
    IR_pix  = get_entry('plate scale, [mas/pix]', ids)
    IR_RON  = get_entry('RON, [e-]', ids)

    M1_area = (8-1.12)**2 * np.pi / 4

    IR_ph = IR_ph / IR_freq * M1_area # [photons/frame/aperture]

    return get_TT_jitter(r0, IR_win, IR_ph, IR_pix, IR_RON) # array of [mas]

Let's just select some random PSFs and compute $J_x$ and $J_y$ for them:

In [17]:
print("Here comes the jitter, [mas]:\n")
ids = np.random.choice(muse_df.index, 10)
jitter = get_jitter_from_ids(ids)

for i, id in enumerate(ids):
    if not np.isnan(jitter[i]):
        print(f"{muse_df.loc[id, 'name']}: {jitter[i]:.2f}")

Here comes the jitter, [mas]:

M.MUSE.2021-01-20T11-11-26.403.fits: 0.14
M.MUSE.2019-08-02T08-36-27.276.fits: 3.29
M.MUSE.2021-12-06T14-57-29.310.fits: 1.34
M.MUSE.2024-05-02T22-51-44.900.fits: 5.28
M.MUSE.2023-08-16T14-31-29.286.fits: 0.82
M.MUSE.2021-03-24T22-04-22.033.fits: 3.07
M.MUSE.2021-03-30T15-04-31.323.fits: 0.21
M.MUSE.2023-05-06T19-13-04.533.fits: 0.49
M.MUSE.2022-09-21T22-17-32.573.fits: 1.11
M.MUSE.2022-10-26T15-34-33.593.fits: 6.20


As you can see, the values are violently underpredicted. I decided to leave it as it is and just switched to the data-driven calibrator.
If you find out some rrors, I will greatly appreciate it.